## [7주차 과제] 통제집단합성법

해당 데이터는 영아사망률 감소에 있어 NGO의 구호활동의 효과를 측정하기 위해 생성한 가상의 결과입니다.
이번 과제의 목표는 통제집단합성법의 쓰임새를 이해하는 것입니다.


---
**과제 가이드**

'date': 날짜 (월별)
'city': 도시
'treated': 구호활동(처치) 여부
'ratio': 영아사망률

1. ATT 추정의 측면에서 수평 회귀 분석, 표준 통제집단합성법, 합성 이중차분법의 차이를 확인해봅니다.
2. 처치 이전 데이터가 충분히 확보되었는지 여부에 따라 추정 결과에 차이가 생기는지 확인해봅니다.

---


과제 제출 기한: 2024년 7월 11일(목) 19:00<br>
제출 방법: `#24s-causal-inference` Slack 채널에 작성한 Notion 링크 공유<br>

*출제자: 권정연, 최목원 <br>
최종 수정일: 2024년 7월 8일(월) 12:00*



### 1. 라이브러리 세팅

In [5]:
from toolz.curried import *


import pandas as pd
import numpy as np

import statsmodels.formula.api as smf

import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib

from cycler import cycler

color=['0.0', '0.4', '0.8']
default_cycler = (cycler(color=color))
linestyle=['-', '--', ':', '-.']
marker=['o', 'v', 'd', 'p']

plt.rc('axes', prop_cycle=default_cycler)

### 2. 데이터 준비

데이터 준비과정에서는 , 데이터를 불러오고 실험군을 설정합니다.

그리고 post열을 생성하여 실험군의 평균과 대조군의 평균을 그래프로 나타내봅니다.

- 데이터 불러오기

In [2]:
import pandas as pd
import numpy as np

df = (pd.read_csv("7thweek_data.csv")
      .astype({"date":"datetime64[ns]"})
      .drop('Unnamed: 0', axis=1))

df.head()

FileNotFoundError: [Errno 2] No such file or directory: '7thweek_data.csv'

- 실험군 설정하기

In [ ]:
# 필요한 정보
cities = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
tr_period = df[df['treated']==1]["date"].min()
treated = list('ABC')

- post 열 만들기

In [ ]:
# post 열 만들기
df['post'] = (_______).astype(int)
df.head()

- 실험군의 평균과 대조군의 평균 그래프 생성

In [ ]:
np.random.seed(123)

df_sc = df.pivot(index= "date",
                 columns = "city",
                 values = "ratio")

ax = df_sc[treated].mean(axis=1).plot(figsize=(15,6), label="$E[Y|D=1]$")
ax.vlines(tr_period, 0.04, 0.11, ls="dashed", label="$t=T_{pre}+1$")
ax.legend()
df_sc.drop(columns=treated).sample(frac=0.2, axis=1).plot(color="0.5", alpha=0.2, legend=False, ax=ax)
plt.ylabel("ratio")

개입 이후 영아 사망률이 감소하는 것을 볼 수 있지만, 이것이 과연 처치에 의해서인지는 명확하지 않습니다.

따라서 먼저 반사실을 추정해야 합니다.

In [ ]:
df_sc = df.pivot(index= "date",
                 columns = "city",
                 values = "ratio")
df_sc

### 3. 통제 집단 합성법과 수평 회귀 분석

데이터에 대해 잘 살펴보았으니, 이제는 본격적으로 통제집단 합성법에 관해 알아봐야겠죠?

먼저 수평 회귀 분석을 통해 통제 집단을 합성해봅시다.<br>

이때, 처치 이전의 데이터가 제한적인 경우 합성 결과가 어떻게 달라질까요? 주어진 데이터와 제한적인 데이터를 비교하여 분석을 진행합니다.

1 ) 처치 이전의 기간이 긴 경우

- 행렬 표현

In [ ]:
def reshape_sc_data(df: pd.DataFrame,
                    geo_col: str,
                    time_col: str,
                    y_col: str,
                    tr_geos: str,
                    tr_start: str):

    df_pivot = df.pivot(index = time_col,
                        columns = geo_col,
                        values = y_col)

    y_co = df_pivot.drop(columns=tr_geos)
    y_tr = df_pivot[tr_geos]

    y_pre_co = y_co[df_pivot.index < tr_start]
    y_pre_tr = y_tr[df_pivot.index < tr_start]

    y_post_co = y_co[df_pivot.index >= tr_start]
    y_post_tr = y_tr[df_pivot.index >= tr_start]

    return y_pre_co, y_pre_tr, y_post_co, y_post_tr

In [ ]:
#4종류의 행렬 만들기
y_pre_co, y_pre_tr, y_post_co, y_post_tr = reshape_sc_data(
    df,
    geo_col="city",
    time_col="date",
    y_col="ratio",
    tr_geos=treated,
    tr_start=str(tr_period)
)

#실험군 행렬, 대조군 행렬 만들기
y_co = pd.concat(___)
y_tr = pd.concat(___)

#y_pre_tr.head()

- 수평 회귀분석을 통한 합성통제집단 만들기

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression(fit_intercept=False)
model.fit(___, ___)

# 가중치 추출
weights_lr = ___
weights_lr.round(3)

In [ ]:
y0_tr_hat = model.predict(___)
y0_tr_hat

- 실험군의 평균과 합성통제집단의 추정 평균 그래프 생성

In [ ]:
plt.figure(figsize=(10,4))

plt.plot(y_co.index, model.predict(___), label="Synthetic Control", ls="-.")
plt.plot(y_tr.mean(axis=1), label="Treated Avg.")
plt.vlines(pd.to_datetime("2015-01-01"), 0, 0.15, ls="dashed", label="$t=T_{pre}+1$")
plt.xticks(rotation=45)
plt.ylabel("ratio")

- 실험군의 평균과 합성통제집단의 추정 평균을 이용해 ATT 그래프 생성

In [ ]:
att = ____________

plt.figure(figsize=(10,4))
#ATT 그래프
plt.plot(___ - model.predict(___), label="ATT", ls="-.")
plt.vlines(pd.to_datetime(tr_period), att.min(), att.max(), ls="dashed", label="$t=T_{pre}+1$")
plt.hlines(0, y_tr.index.min(), y_tr.index.max())
plt.ylabel("ATT")
plt.xticks(rotation=45)
plt.legend()

**Q1. 위에 적은 ATT 식을, ATT의 의미를 고려해 설명해주세요.**

2 ) 처치 이전의 기간이 짧은 경우

- 행렬 표현

In [ ]:
#처치 이전 데이터가 2013-01-01부터 주어진다고 가정
df_2 = df[df['date']>=pd.Timestamp(___)]

y_pre_co_2, y_pre_tr_2, y_post_co_2, y_post_tr_2 = reshape_sc_data(
    df_2,
    geo_col="city",
    time_col="date",
    y_col="ratio",
    tr_geos=treated,
    tr_start=str(tr_period)
)

y_co_2 = pd.concat(___)
y_tr_2 = pd.concat(___)

In [ ]:
y_pre_tr_2.head()

- 수평 회귀분석을 통한 합성통제집단 만들기

In [ ]:
from sklearn.linear_model import LinearRegression

model_2 = LinearRegression(fit_intercept=False)
model_2.fit(___, ___)

# extract the weights
weights_lr_2 = model_2.___
weights_lr_2

In [ ]:
y0_tr_hat_2 = model_2.predict(___)
y0_tr_hat_2

In [ ]:
plt.figure(figsize=(10,4))

plt.plot(y_co_2.index, model_2.predict(___), label="Synthetic Control", ls="-.")
plt.plot(y_tr_2.mean(axis=1), label="Treated Avg.")
plt.vlines(pd.to_datetime("2015-01-01"), 0, 0.15, ls="dashed", label="$t=T_{pre}+1$")
plt.xticks(rotation=45)
plt.ylabel("ratio")

- 실험군의 평균과 합성통제집단의 평균을 이용해 ATT 그래프 생성

In [ ]:
att_2 = ____________

plt.figure(figsize=(10,4))
#ATT 그래프
plt.plot(___ - model_2.predict(___), label="ATT", ls="-.")
plt.vlines(pd.to_datetime(tr_period), att_2.min(), att_2.max(), ls="dashed", label="$t=T_{pre}+1$")
plt.hlines(0, y_tr_2.index.min(), y_tr_2.index.max())
plt.ylabel("ATT")
plt.xticks(rotation=45)
plt.legend()


**Q2. 처치 이전 데이터가 짧아졌을 때 ATT 추정에 있어 달라진 점이 있나요? 있다면 무엇이고, 없다면 이유가 무엇이라고 생각하나요?**

### 4. 표준 통제집단합성법

1 ) 처치 이전의 기간이 긴 경우

In [ ]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import (check_X_y, check_array,
                                      check_is_fitted)
import cvxpy as cp

class SyntheticControl(BaseEstimator, RegressorMixin):

    def __init__(self,):
        pass

    def fit(self, y_pre_co, y_pre_tr):

        y_pre_co, y_pre_tr = check_X_y(y_pre_co, y_pre_tr)

        w = cp.Variable(y_pre_co.shape[1])

        objective = cp.Minimize(cp.sum_squares(y_pre_co@w - y_pre_tr))
        constraints = [cp.sum(w) == 1, w >= 0]

        problem = cp.Problem(objective, constraints)

        self.loss_ = problem.solve(verbose=False)
        self.w_ = w.value

        self.is_fitted_ = True
        return self


    def predict(self, y_co):

        check_is_fitted(self)
        y_co = check_array(y_co)

        return y_co @ self.w_

In [ ]:
# 모델 적합하기
model = SyntheticControl()
model.fit(___, ___)

# 가중치 추정하기
model.___.round(3)

array([ 0.018,  0.054,  0.   ,  0.059,  0.088,  0.116, -0.   , -0.   ,
        0.067,  0.07 ,  0.104,  0.051,  0.002,  0.025,  0.034,  0.103,
       -0.   ,  0.077, -0.   , -0.   ,  0.083,  0.049, -0.   ])

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15,8), sharex=True)

ax1.plot(y_co.index, model.predict(___), label="Synthetic Control")
ax1.plot(y_tr.mean(axis=1), label="Treated", ls="-.")
ax1.vlines(pd.to_datetime(tr_period), 0.04, 0.09, ls="dashed", label="$T_0$")
ax1.legend()
ax1.set_ylabel("ATT")

ax2.plot(y_co.index, ___ - model.predict(___), label="Effect", ls="-.")
ax2.hlines(0, y_co.index.min(), y_co.index.max())
ax2.vlines(pd.to_datetime(tr_period), -0.03, 0.02, label="$T_0$")
ax2.legend()
ax2.set_ylabel("ratio")

plt.xticks(rotation=45);

**Q3. 본 데이터에서 수평 회귀 분석이 아닌 표준 통제집단합성법을 활용했을 때 어떠한 차이가 발생했나요? 본 데이터를 기준으로 표준 통제집단합성법 활용의 의의를 설명해주세요.**

2 ) 처치 이전의 기간이 짧은 경우

In [ ]:
model2 = SyntheticControl()
model2.fit(___, ___)

# extrac the weights
model2.___.round(3)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15,8), sharex=True)

ax1.plot(y_co_2.index, model2.predict(___), label="Synthetic Control")
ax1.plot(y_tr_2.mean(axis=1), label="Treated", ls="-.")
ax1.vlines(pd.to_datetime(tr_period), 0.04, 0.09, ls="dashed", label="$T_0$")
ax1.legend()
ax1.set_ylabel("ATT")

ax2.plot(y_co_2.index, ___ - ___, label="Effect", ls="-.")
ax2.hlines(0, y_co_2.index.min(), y_co_2.index.max())
ax2.vlines(pd.to_datetime(tr_period), -0.03, 0.02, label="$T_0$")
ax2.legend()
ax2.set_ylabel("ratio")

plt.xticks(rotation=45);

**Q4. 처치 이전 데이터가 짧아졌을 때 표준통제집단합성법을 활용한 ATT 추정에 있어 달라진 점이 있나요? 있다면 무엇이고, 없다면 이유가 무엇이라고 생각하나요?**

### 5. 편향 제거

1 ) 처치 이전의 기간이 긴 경우

In [ ]:
def debiased_sc_atts(y_pre_co, y_pre_tr, y_post_co, y_post_tr, K=3):

    block_size = int(min(np.floor(len(y_pre_tr)/K), len(y_post_tr)))
    blocks = np.split(y_pre_tr.index[-K*block_size:], K)

    def fold_effect(hold_out):
        model = SyntheticControl()
        model.fit(
            y_pre_co.drop(hold_out),
            y_pre_tr.drop(hold_out)
        )

        bias_hat = np.mean(y_pre_tr.loc[hold_out]
                           - model.predict(y_pre_co.loc[hold_out]))

        y0_hat = model.predict(y_post_co)
        return (y_post_tr - y0_hat) - bias_hat


    return pd.DataFrame([fold_effect(block) for block in blocks]).T

In [ ]:
deb_atts = debiased_sc_atts(y_pre_co,
                            y_pre_tr.mean(axis=1),
                            y_post_co,
                            y_post_tr.mean(axis=1),
                            K=3)

deb_atts.head()

In [ ]:
model_biased = SyntheticControl()
model_biased.fit(___, ___)
att = ____________
att_biased = ____________

plt.figure(figsize=(10,4))
plt.plot(att, label="ATT Debiased", ls="-.")
plt.plot(att_biased, label="ATT Biased")
plt.legend()
plt.hlines(0, att.index.min(), att.index.max())
plt.ylabel("ATT")
plt.xticks(rotation=45);

2 ) 처치 이전의 기간이 짧은 경우

In [ ]:
deb_atts_2 = debiased_sc_atts(y_pre_co_2,
                            y_pre_tr_2.mean(axis=1),
                            y_post_co_2,
                            y_post_tr_2.mean(axis=1),
                            K=3)

deb_atts_2.head()

In [ ]:
model_biased_2 = SyntheticControl()
model_biased_2.fit(y_pre_co_2, y_pre_tr_2.mean(axis=1))

att_biased_2 = y_post_tr_2.mean(axis=1) - model_biased_2.predict(y_post_co_2)

att_2 = y_post_tr_2.mean(axis=1) - y0_tr_hat_2

plt.figure(figsize=(10,4))
plt.plot(att_2, label="ATT Debiased", ls="-.")
plt.plot(att_biased_2, label="ATT Biased")
plt.legend()
plt.hlines(0, att_2.index.min(), att_2.index.max())
plt.ylabel("ATT")
plt.xticks(rotation=45);

**Q5. 전체 데이터와 부분 데이터에서 편향을 제거한 후의 ATT 추정값을 기존 추정값과 비교해봤습니다. 두 데이터에서 각각 편향 제거 전후 결과에 어떤 차이가 발생했나요? 해당 결과에 대한 이유를 자유롭게 적어주세요.**

### 6. 통제집단합성법과 이중차분법

1 ) 처치 이전의 기간이 긴 경우

- 통제 집단 합성법을 추정 후 ATT 계산

In [ ]:
sc_model = SyntheticControl()
sc_model.fit(___, ___)

att = _______________

- 대조군 가중치 데이터프레임에 추가

In [ ]:
unit_w = pd.DataFrame(zip(y_pre_co.columns, sc_model.___),
                         columns=["city", "unit_weight"])

unit_w.head()

- 가중치 데이터 프레임을 원래 데이터프레임에 조인

In [ ]:
df_with_w = (df
             .assign(tr_post = lambda d: d["post"]*d["treated"])
             .merge(unit_w, on=["city"], how="left")
             .fillna({"unit_weight": df["treated"].mean()}))

df_with_w.head()

- 가중 OLS 활용

In [ ]:
mod = smf.wls(
    "_________",
    data=df_with_w.query("____________"),
    weights=df_with_w.query("_________")["unit_weight"]
)

mod.fit()._________

**Q6. 시간 가중치 중 일부만 골라 사용하는 이유가 무엇일까요? 자유롭게 적어주세요.**

- 시간가중치 추정

In [ ]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
import cvxpy as cp

class SyntheticControl(BaseEstimator, RegressorMixin):

    def __init__(self, fit_intercept=False):
        self.fit_intercept = fit_intercept

    def fit(self, y_pre_co, y_pre_tr):

        y_pre_co, y_pre_tr = check_X_y(y_pre_co, y_pre_tr)

        # add intercept
        intercept = np.ones((y_pre_co.shape[0], 1))*self.fit_intercept
        X = np.concatenate([intercept, y_pre_co], axis=1)
        w = cp.Variable(X.shape[1])

        objective = cp.Minimize(cp.sum_squares(X@w - y_pre_tr))
        constraints = [cp.sum(w[1:]) == 1, w[1:] >= 0]

        problem = cp.Problem(objective, constraints)

        self.loss_ = problem.solve(verbose=False)
        self.w_ = w.value[1:] # remove intercept

        self.is_fitted_ = True
        return self


    def predict(self, y_co):

        check_is_fitted(self)
        y_co = check_array(y_co)

        return y_co @ self.w_

In [ ]:
time_sc = SyntheticControl(fit_intercept=True)

time_sc.fit(___, ___)

time_w = pd.DataFrame(zip(y_pre_co.index, time_sc.___),
                         columns=["date", "time_weight"])

time_w.tail()

In [ ]:
plt.figure(figsize=(10,4))
plt.bar(time_w["date"], time_w["time_weight"])
plt.ylabel("Time Weights")
plt.xticks(rotation=45);

- 합성 이중차분법을 이용해 ATT 추정값 구하기

In [ ]:
scdid_df = (
    df_with_w
    .merge(time_w, on=["date"], how="left")
    .fillna({"time_weight":df["post"].mean()})
    .assign(weight = lambda d: (d["time_weight"]*d["unit_weight"]))
)

In [ ]:
did_model = smf.wls(
    "_________",
    data=scdid_df.query("______"),
    weights=scdid_df.query("______")["weight"]).fit()

did_model.params["treated:post"]

In [ ]:
avg_pre_period = time_w.iloc[int((time_w["time_weight"] * np.arange(0, len(time_w), 1)).sum().round())]["date"]
avg_post_period = pd.Timestamp("2022-06-01")

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15,8), sharex=True, gridspec_kw={'height_ratios': [3, 1]})

# 실험집단의 평균 영아사망률 그래프 그리기
ax1.plot(y_co.index, ___, label="Treated")
# 합성이중차분법 통한 추정값 그래프 그리기
ax1.plot(y_co.index, sc_model.predict(___), label="Synthetic Control", ls="-.")
ax1.vlines(tr_period, y_tr.mean(axis=1).min(), y_tr.mean(axis=1).max(), color="black", ls="dotted")

pre_sc = ______
post_sc = ______
pre_treat = ______
post_treat = ______

sc_did_y0 = ______

ax1.plot([avg_pre_period, avg_post_period], [pre_treat, post_treat], color="C2", ls="dashed")
ax1.plot([avg_pre_period, avg_post_period], [pre_treat, sc_did_y0], color="C2")
ax1.legend()
ax1.set_title("Synthetic Diff-in-Diff")
ax1.set_ylabel("app_download_pct")

ax2.bar(time_w["date"], time_w["time_weight"])
ax2.vlines(tr_period, 0, 0.15, color="black", ls="dotted")
ax2.set_ylabel("Time Weights")
ax2.set_xlabel("Dates")
plt.xticks(rotation=30);

- 표준 통제집단합성법 vs. 합성 이중차분법

In [ ]:
#att_unbiased = ______ (remind ^^)
att = ________________________
att_synthetic = ________________________

plt.figure(figsize=(10,4))
plt.plot(att, label="ATT standard", ls="-.")
plt.plot(att_synthetic, label="ATT synthetic")
plt.legend()
plt.hlines(0, att.index.min(), att.index.max())
plt.ylabel("ATT")
plt.xticks(rotation=45);

**Q7. 합성 이중 차분법을 활용하여 ATT 추정값을 구했을 때와 표준 통제집단합성법으로 구한 ATT 추정값에는 어떤 차이가 있나요?**

2 ) 처치 이전의 기간이 짧은 경우

- 통제 집단 합성법을 추정 후 ATT 계산

In [ ]:
sc_model_2 = SyntheticControl()
sc_model_2.fit(______, ______)
att = ________________________

- 대조군 가중치 데이터프레임에 추가

In [ ]:
unit_w_2 = pd.DataFrame(zip(y_pre_co_2.columns, sc_model_2.___),
                         columns=["city", "unit_weight"])
unit_w_2.head()

- 가중치 데이터 프레임을 원래 데이터프레임에 조인

In [ ]:
df_with_w_2 = (df_2
             .assign(tr_post = lambda d: d["post"]*d["treated"])
             .merge(unit_w_2, on=["city"], how="left")
             .fillna({"unit_weight": df_2["treated"].mean()}))
df_with_w_2.head()

- 가중 OLS 활용

In [ ]:
mod_2 = smf.wls(
    "_________",
    data=df_with_w_2.query("______"),
    weights=df_with_w_2.query("______")["unit_weight"]
)
mod_2.fit()._________

- 시간 가중치 추정

In [ ]:
time_sc_2 = SyntheticControl(fit_intercept=True)

time_sc_2.fit(___, ___)

time_w_2 = pd.DataFrame(zip(y_pre_co_2.index, time_sc_2.w_),
                         columns=["date", "time_weight"])
time_w_2.tail()

- 합성 이중차분법을 이용해 ATT 추정값 구하기

In [ ]:
scdid_df_2 = (
    df_with_w_2
    .merge(time_w_2, on=["date"], how="left")
    .fillna({"time_weight":df_2["post"].mean()})
    .assign(weight = lambda d: (d["time_weight"]*d["unit_weight"]))
)

In [ ]:
did_model_2 = smf.wls(
    "_________",
    data=scdid_df_2.query("______"),
    weights=scdid_df_2.query("______")["weight"]).fit()
did_model_2._________

In [ ]:
avg_pre_period = time_w_2.iloc[int((time_w_2["time_weight"] * np.arange(0, len(time_w_2), 1)).sum().round())]["date"]
avg_post_period = pd.Timestamp("2022-06-01")

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15,8), sharex=True, gridspec_kw={'height_ratios': [3, 1]})

# 실험집단의 평균 영아사망률 그래프 그리기
ax1.plot(y_co_2.index, ___, label="Treated")
# 합성이중차분법 통한 추정 ATT 그래프 그리기
ax1.plot(y_co_2.index, sc_model_2.predict(___), label="Synthetic Control", ls="-.")
ax1.vlines(tr_period, y_tr_2.mean(axis=1).min(), y_tr_2.mean(axis=1).max(), color="black", ls="dotted")

pre_sc = ______
post_sc = ______
pre_treat = ______
post_treat = ______

sc_did_y0 = ______

ax1.plot([avg_pre_period, avg_post_period], [pre_treat, post_treat], color="C2", ls="dashed")
ax1.plot([avg_pre_period, avg_post_period], [pre_treat, sc_did_y0], color="C2")
ax1.legend()
ax1.set_title("Synthetic Diff-in-Diff")
ax1.set_ylabel("app_download_pct")

ax2.bar(time_w_2["date"], time_w_2["time_weight"])
ax2.vlines(tr_period, 0, 0.15, color="black", ls="dotted")
ax2.set_ylabel("Time Weights")
ax2.set_xlabel("Dates")
plt.xticks(rotation=30);

- 표준 통제집단합성법 vs. 합성 이중차분법

In [ ]:
#att_unbiased = ______ (remind ^^)
att_2 = ________________________
att_synthetic_2 = ________________________

plt.figure(figsize=(10,4))
plt.plot(att_2, label="ATT standard", ls="-.")
plt.plot(att_synthetic_2, label="ATT synthetic")
plt.legend()
plt.hlines(0, att_2.index.min(), att_2.index.max())
plt.ylabel("ATT")
plt.xticks(rotation=45);

**Q8. 전체 데이터와 부분 데이터에서 합성 이중차분법의 ATT 추정값을 각각 구했습니다. 두 데이터에서 합성 이중차분법의 사용은 각각 효과적이라고 생각하나요? 판단에 대한 이유를 자유롭게 적어주세요.**